In [1]:
!pip install allennlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 730.2/730.2 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.3/776.3 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 48.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.2/248.2 kB 25.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 116.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 120.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 90.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 37.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

In [2]:
import numpy as np
import pandas as pd
import torch
from allennlp.modules.elmo import Elmo, batch_to_ids
import jieba

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
cd /content/drive/MyDrive/text_classification/dot/word2vec-elmo

/content/drive/MyDrive/text_classification/dot/word2vec-elmo


In [5]:
## elmo ##
# Read the Excel file
data = pd.read_excel('./data.xlsx')
sentences = data['review'].tolist()

# Load ELMo model
options_file = "https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/2x4096_512_2048cnn_2xhighway/elmo_2x4096_512_2048cnn_2xhighway_options.json"
weight_file = "https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/2x4096_512_2048cnn_2xhighway/elmo_2x4096_512_2048cnn_2xhighway_weights.hdf5"
elmo_model = Elmo(options_file, weight_file, num_output_representations=1)

word_vectors = []

# Define a function to tokenize using jieba
def tokenize(text):
    return " ".join(jieba.cut(text))

for sentence in sentences:
    tokens = [tokenize(sentence) for sentence in sentences]
    character_ids = batch_to_ids([tokens])

    with torch.no_grad():
        embeddings = elmo_model(character_ids)['elmo_representations'][0]

    word_vector = embeddings.squeeze(0).cpu().numpy()
    word_vectors.append(word_vector)

Output()

Output()

Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
DEBUG:jieba:Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.578 seconds.
DEBUG:jieba:Loading model cost 0.578 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.


In [ ]:
# 遍历句子和单词
for i, sentence in enumerate(sentences):
    tokens = [tokenize(sentence)]
    character_ids = batch_to_ids(tokens)

    with torch.no_grad():
        embeddings = elmo_model(character_ids)['elmo_representations'][0]

    # 获取句子中每个单词的ELMo词向量
    for j, token in enumerate(tokens[0].split()):
        word_vector = embeddings[0, j].cpu().numpy()
        print(f"Word: {token}, ELMo Vector: {word_vector}, shape: {word_vector.shape}")

In [8]:
import pandas as pd
from gensim.models import Word2Vec
import jieba

In [ ]:
## word2vec ##
def txt_cut(s):
    res = [w for w in jieba.lcut(s) if w.strip()]
    return " ".join(res)

# 对文本进行分词
tokenized_text = []
for i in sentences:
    new = txt_cut(i).split(' ')
    tokenized_text.extend(new)

# 初始化Word2Vec模型
model_w = Word2Vec(vector_size=100, window=5, min_count=1)

# 构建词汇表
model_w.build_vocab([tokenized_text])

# 训练模型
model_w.train([tokenized_text], total_examples=model_w.corpus_count, epochs=model_w.epochs)

# 创建一个空的DataFrame来保存词汇和对应的词向量
word_vector_df = pd.DataFrame()

for wd in model_w.wv.index_to_key:
    # 将词向量转化为Series，然后添加到DataFrame中
    word_vector_series = pd.Series([wd] + list(model_w.wv[wd]))
    word_vector_df = word_vector_df.append(word_vector_series, ignore_index=True)

In [12]:
new_word_vectors = []

# 遍历句子和单词
for i, sentence in enumerate(sentences):
    tokens = [tokenize(sentence)]
    character_ids = batch_to_ids(tokens)

    with torch.no_grad():
        embeddings = elmo_model(character_ids)['elmo_representations'][0]

    # 获取句子中每个单词的ELMo词向量
    for j, token in enumerate(tokens[0].split()):
        word_vector_e = embeddings[0, j].cpu().numpy()
        word_vector_e = word_vector_e.reshape(-1, 1)

        ## word2vec
        word_vector_w = model_w.wv[token]

        word_vector_w = word_vector_w.reshape(-1, 1)

        ## 新词向量
        new_word_vector = np.multiply(word_vector_e, word_vector_w.T)

        new_word_vectors.append((token, new_word_vector))

In [15]:
# 指定保存文件路径
save_path = 'new_word_vectors.txt'

# 保存单词和新词向量到文件
with open(save_path, 'w') as f:
    for word, word_vector in new_word_vectors:
        word_vector_str = ','.join(map(str, word_vector))
        line = f"{word}: {word_vector_str}\n"
        f.write(line)